In [ ]:
# ============================================================================================
# Kassio Machado - GNU Public License - 2016-05-22
# PhD candidate on Science Computing - UFMG/Brazil
# Jupyter Notebook to collect data from Weather Underground API
# Require accounts for authentication
# ============================================================================================ 
import time
import json
import urllib2
import pymongo
import colorama
import datetime
from tqdm import tqdm_notebook as tqdm
from pymongo import MongoClient
info = '[INFO]'
print info, 'Initializing the Modules: [OK]' 

In [ ]:
# environment variables
selectedCities = [('new-york', 'NY', 'New_York'), ('los-angeles', 'CA', 'Los_Angeles'), 
                  ('chicago', 'IL', 'Chicago'), ('london', 'England', 'London'), 
                  ('paris', 'France', 'Paris'), ('berlin', 'Germany', 'Berlin')]
authKeys = ['1', '2', '3']

# database parameters
dbPass = 'npng'
dbLogin = 'leokassio'
dbAuthentication = 'admin'
dbaddress = 'localhost'
dateBegin = datetime.datetime.strptime('2015-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
dateEnd = datetime.datetime.strptime('2015-12-31 23:59:59', '%Y-%m-%d %H:%M:%S')
dateWuFormat = '%Y%m%d'
dateFormat = '%Y-%m-%d'
dateTimeFormat = '%Y-%m-%d %H:00:00'
print info, 'Defining variables: [OK]'

In [ ]:
# database connection
con = MongoClient(dbaddress)
con.sensing_layers.authenticate(dbLogin, dbPass, source=dbAuthentication)
database = con.sensing_layers
wunderground = database.weather_underground
print info, 'Connecting database: [OK]'

In [ ]:
lastPrint = None
keyAuthIndex = 3
keyAuth = authKeys[keyAuthIndex]
cityName, majorWuIndex, minorWuIndex = selectedCities[0]
dateRequest = dateBegin
nDays = (dateEnd - dateBegin).days

print info, 'Please Check Starting Date and Authorization Key'
print info, cityName, 'DateBegin:', dateRequest.strftime(dateTimeFormat), 'KeyAuth:', keyAuthIndex, keyAuth
ans = raw_input('Press any key to continue >>')
if ans != '':
    exit()       
    
while dateRequest <= dateEnd:
    dateKey = dateRequest.strftime(dateWuFormat)
    url = 'http://api.wunderground.com/api/'
    url += keyAuth + '/history_' + dateKey
    url += '/q/' + majorWuIndex + '/' + minorWuIndex + '.json'
    weekNumber = dateRequest.isocalendar()[1]
    if lastPrint != weekNumber:
        lastPrint = weekNumber
        print 'Fetching:', url
    try:
        f = urllib2.urlopen(url) 
        json_string = f.read()
        parsed_json = json.loads(json_string)
    except Exception, e:
        print info, 'Error on Fetching WUnderground API'
        print str(e)
        time.sleep(60)
        continue
        
    try:
        dateStr = parsed_json['history']['date']['pretty']
        observations = parsed_json['history']['observations']
        for observ in observations:
            d = datetime.datetime(year=int(observ['date']['year']), month=int(observ['date']['mon']),
                                  day=int(observ['date']['mday']), hour=int(observ['date']['hour']), 
                                  minute=int(observ['date']['min']), second=0)
            observ['date_key'] = d.strftime(dateFormat)
            observ['date_time_key'] = d.strftime(dateTimeFormat)
            observ['date_original'] = observ['date']
            observ['date_local'] = d
            observ['tempm'] = observ['tempm']
            observ['city'] = cityName
            del observ['date']
            del observ['utcdate']
            del observ['metar']
            uid = wunderground.insert_one(observ)
    except KeyError, e:
        print info, 'Error on Processing JSON file - probably API limit'
        print str(e)
        time.sleep(60)
    time.sleep(7)
    dateRequest  = dateRequest + datetime.timedelta(days=1)
        